# Packages and Helper Functions

In [112]:
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd
import numpy as np
import polars as pl
import scipy.stats as stats
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import math

import requests
import re
import pdfplumber

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: "%.4f" % x)
# pd.options.plotting.backend = "plotly"

plt.style.use('ggplot')
sns.set_style('darkgrid')

# Reading in Data

## Pulling PDFs

In [113]:
isu_year = 1718
isu_event = "owg2018"
dir_name = f"./{isu_event}"
isu_url = "https://www.isuresults.com/results/season1718/owg2018/"
OUTPUT_CSV = dir_name + f"/{isu_event}.csv"

# Get the HTML content of the page
response = requests.get(isu_url)
html_content = response.text

# find all pdf extension files
regex_pattern = r'href=([^\s>]+\.pdf)'

# Find all matches
matches = re.findall(regex_pattern, html_content, re.IGNORECASE)

print(matches)
print(len(matches))

['OWG2018_MenSingleSkating_SP_Scores.pdf', 'OWG2018_MenSingleSkating_FS_Scores.pdf', 'OWG2018_LadiesSingleSkating_SP_Scores.pdf', 'OWG2018_LadiesSingleSkating_FS_Scores.pdf', 'OWG2018_PairSkating_SP_Scores.pdf', 'OWG2018_PairSkating_FS_Scores.pdf', 'OWG2018_IceDance_SD_Scores.pdf', 'OWG2018_IceDance_FD_Scores.pdf', 'TeamEntries.pdf', 'OWG2018_TeamMen_SP_M_Scores.pdf', 'OWG2018_TeamMen_FS_M_Scores.pdf', 'OWG2018_TeamLadies_SP_F_Scores.pdf', 'OWG2018_TeamLadies_FS_F_Scores.pdf', 'OWG2018_TeamPairs_SP_P_Scores.pdf', 'OWG2018_TeamPairs_FS_P_Scores.pdf', 'OWG2018_TeamIceDance_SD_D_Scores.pdf', 'OWG2018_TeamIceDance_FD_D_Scores.pdf']
17


In [114]:
# find all score sheets in the pdf files
score_sheet_file_names = [m for m in matches if "scores" in m.lower()]
score_sheet_urls = [isu_url + m for m in score_sheet_file_names]
print(score_sheet_urls)
print(len(score_sheet_urls))

['https://www.isuresults.com/results/season1718/owg2018/OWG2018_MenSingleSkating_SP_Scores.pdf', 'https://www.isuresults.com/results/season1718/owg2018/OWG2018_MenSingleSkating_FS_Scores.pdf', 'https://www.isuresults.com/results/season1718/owg2018/OWG2018_LadiesSingleSkating_SP_Scores.pdf', 'https://www.isuresults.com/results/season1718/owg2018/OWG2018_LadiesSingleSkating_FS_Scores.pdf', 'https://www.isuresults.com/results/season1718/owg2018/OWG2018_PairSkating_SP_Scores.pdf', 'https://www.isuresults.com/results/season1718/owg2018/OWG2018_PairSkating_FS_Scores.pdf', 'https://www.isuresults.com/results/season1718/owg2018/OWG2018_IceDance_SD_Scores.pdf', 'https://www.isuresults.com/results/season1718/owg2018/OWG2018_IceDance_FD_Scores.pdf', 'https://www.isuresults.com/results/season1718/owg2018/OWG2018_TeamMen_SP_M_Scores.pdf', 'https://www.isuresults.com/results/season1718/owg2018/OWG2018_TeamMen_FS_M_Scores.pdf', 'https://www.isuresults.com/results/season1718/owg2018/OWG2018_TeamLadies

In [115]:
# create data store
# if dir_name exists, skip this
if os.path.isdir(dir_name):
    print(f"data directory already exists: {dir_name}")
else:
    print(f"making data directory: {dir_name}")
    os.makedirs(dir_name, exist_ok=True)

    for url in score_sheet_urls:
        response = requests.get(url)
        filename = url.split('/')[-1]
        filepath = os.path.join(dir_name, filename)
        with open(filepath, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {filename}")

data directory already exists: ./owg2018


## Regex Parsing of PDFs

Below is testing on 1 file. For complete pipeline, go to Applying Data Pipeline section

In [116]:
PDF_PATH = dir_name + "/OWG2018_MenSingleSkating_SP_Scores.pdf"

# use pdfplumber to get the complete text string of score pdf
pages_text = []
with pdfplumber.open(PDF_PATH) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            pages_text.append(text)

full_text = "\n".join(pages_text)

# regex pattern for finding each skater 
skater_header_pattern = re.compile(
    r"""
    ^(\d+)\s+                  # 1 rank
    (.+?)\s+                   # 2 name
    ([A-Z]{3})\s+              # 3 NOC code
    (\d+)\s+                   # 4 starting num
    (\d+\.\d{2})\s+            # 5 total segment score
    (\d+\.\d{2})\s+            # 6 total element score
    (\d+\.\d{2})\s+            # 7 total program score
    (-?\d+\.\d{2})$            # 8 total deductions
    """,
    re.VERBOSE | re.MULTILINE
)

# regex pattern for finding each skater's element
element_pattern = re.compile(
    r"""
    ^\s*(\d+)\s+                 # 1 element number
    ([A-Za-z0-9+!*<>q]+)\s+      # 2 element code
    (?:(\S+)\s+)?                # 3 optional info column (x, q, !, etc.)
    ([\d.]+)\s+                  # 4 base value
    (?:\b(x)\b\s+)?              # 5 optional extra points column (x)
    ([\-\d.]+)\s+                # 6 GOE
    ((?:(?:-?\d+)|-)(?:\s+(?:(?:-?\d+)|-)){8}\s+)  # 7 judges scores 
    ([\d.]+)$                    # 8 final score
    """,
    re.VERBOSE | re.MULTILINE
)

# regex pattern for finding each skater's program component
program_components_pattern = re.compile(
    r"""
    ^(Skating\s+Skills|Transitions|Performance|Composition|Interpretation\s+of\s+the\s+Music)\s+  # 1 component
    (\d+\.\d{2})\s+                     # 2 factor
    ((?:\d+\.\d{2}\s+){9})              # 3 judge scores
    (\d+\.\d{2})$                       # 4 final score
    """,
    re.VERBOSE | re.MULTILINE
)


In [117]:
# find all skater header matches
matches = list(skater_header_pattern.finditer(full_text))

# for each skater header match, get the complete text between next match
skater_blocks = []
for i, m in enumerate(matches):
    start = m.start()
    end = matches[i + 1].start() if i + 1 < len(matches) else len(full_text)
    skater_blocks.append((m, full_text[start:end]))

# create elements df and program components df row by row
elements_rows = []
program_rows = []

for header, block in skater_blocks:
    # pull skater info
    rank = int(header.group(1))
    name = header.group(2).title()
    noc = header.group(3)
    starting_number = int(header.group(4))
    tss = float(header.group(5))
    tes = float(header.group(6))
    tpcs = float(header.group(7))
    deductions = float(header.group(8))

    for m in element_pattern.finditer(block):
        row = {
            "rank": rank,
            "name": name,
            "noc": noc,
            "starting_number": starting_number,
            "tss": tss,
            "tes": tes,
            "tpcs": tpcs,
            "deductions": deductions,
            "element_no": int(m.group(1)),
            "element": m.group(2),
            "info": m.group(3),
            "base_value": float(m.group(4)),
            "extra_points": 1 if m.group(5) else 0,
            "goe": float(m.group(6)),
            "final_score": float(m.group(8)),
        }

        judges = m.group(7).split()
        for i, j in enumerate(judges):
            try:
                row[f"J{i+1}"] = int(j)
            except ValueError:
                row[f"J{i+1}"] = 0
        
        elements_rows.append(row)
    
    for m in program_components_pattern.finditer(block):
        row = {
            "rank": rank,
            "name": name,
            "noc": noc,
            "starting_number": starting_number,
            "tss": tss,
            "tes": tes,
            "tpcs": tpcs,
            "deductions": deductions,
            "program_component": m.group(1),
            "factor": float(m.group(2)),
            "final_score": float(m.group(4)),
        }

        judges = m.group(3).split()
        for i, j in enumerate(judges):
            try:
                row[f"J{i+1}"] = float(j)
            except ValueError:
                row[f"J{i+1}"] = 0
        
        program_rows.append(row)

element_df = pd.DataFrame(elements_rows)
element_df.sort_values(["rank", "element_no"], inplace=True)

program_df = pd.DataFrame(program_rows)
program_df.sort_values(["rank", "program_component"], inplace=True)


In [118]:
print(full_text)

Gangneung Ice Arena Figure Skating
강릉 아이스 아레나 피겨 스케이팅 / Patinage artistique
Palais des glaces de Gangneung
Men Single Skating
피겨 스케이팅 남자 싱글 / Patinage individuel hommes
FRI 16 FEB 2018 Short Program
쇼트 프로그램 / Programme court
Judges Details per Skater
선수별 심판 세부채점 정보 / Notation détaillée des juges par patineur
Total Total Total Program
NOC Starting Total
Rank Name Segment Element Component Score
Code Number Deductions
Score Score (factored)
1 HANYU Yuzuru JPN 25 111.68 63.18 48.50 0.00
# Executed Elements
FSKMSINGLES-----------QUAL000100--_77B 1.0 Report Created FRI 16 FEB 2018 14:29
ofnI Base Scores
GOE J1 J2 J3 J4 J5 J6 J7 J8 J9 Ref.
Value of Panel
1 4S 10.50 2.71 3 2 3 3 3 2 3 3 2 13.21
2 FCSp4 3.20 1.00 2 2 2 2 2 2 2 3 2 4.20
3 CSSp4 3.00 1.43 2 3 3 3 3 2 3 3 3 4.43
4 3A 9.35 x 3.00 3 3 3 3 3 3 3 3 3 12.35
5 4T+3T 16.06 x 2.57 3 2 2 3 2 2 3 3 3 18.63
6 StSq4 3.90 2.10 2 3 3 3 3 3 3 3 3 6.00
7 CCoSp3 3.00 1.36 2 3 2 3 3 3 2 3 3 4.36
49.01 63.18
Program Components Factor
Skating Skills

### Sense checking element df

In [119]:
print(element_df.shape)

(210, 24)


In [120]:
# in the short, expect each person to do 7 elements
element_df["name"].value_counts()

name
Hanyu Yuzuru                  7
Fernandez Javier              7
Montoya Felipe                7
Martinez Michael Christian    7
Ten Denis                     7
Besseghier Chafik             7
Yee Julian Zhi Jie            7
Fentz Paul                    7
Rizzo Matteo                  7
Kvitelashvili Morisi          7
Vasiljevs Deniss              7
Tanaka Keiji                  7
Yan Han                       7
Samohin Daniel                7
Chen Nathan                   7
Kerry Brendan                 7
Cha Junhwan                   7
Ge Misha                      7
Bychenko Alexei               7
Zhou Vincent                  7
Hendrickx Jorik               7
Messing Keegan                7
Brezina Michal                7
Kolyada Mikhail               7
Rippon Adam                   7
Chan Patrick                  7
Aliev Dmitri                  7
Jin Boyang                    7
Uno Shoma                     7
Paniot Yaroslav               7
Name: count, dtype: int64

In [121]:
# expect 30 skaters in the short
len(element_df["name"].unique())

30

In [122]:
element_df.head()

,rank,name,noc,starting_number,tss,tes,tpcs,deductions,element_no,element,info,base_value,extra_points,goe,final_score,J1,J2,J3,J4,J5,J6,J7,J8,J9
0,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,1,4S,None,10.5000,0,2.7100,13.2100,3,2,3,3,3,2,3,3,2
1,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,2,FCSp4,None,3.2000,0,1.0000,4.2000,2,2,2,2,2,2,2,3,2
2,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,3,CSSp4,None,3.0000,0,1.4300,4.4300,2,3,3,3,3,2,3,3,3
3,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,4,3A,None,9.3500,1,3.0000,12.3500,3,3,3,3,3,3,3,3,3
4,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,5,4T+3T,None,16.0600,1,2.5700,18.6300,3,2,2,3,2,2,3,3,3


### Sense checking program df

In [123]:
print(program_df.shape)

(150, 20)


In [124]:
# expect 5 program components per skater
program_df["name"].value_counts()

name
Hanyu Yuzuru                  5
Fernandez Javier              5
Montoya Felipe                5
Martinez Michael Christian    5
Ten Denis                     5
Besseghier Chafik             5
Yee Julian Zhi Jie            5
Fentz Paul                    5
Rizzo Matteo                  5
Kvitelashvili Morisi          5
Vasiljevs Deniss              5
Tanaka Keiji                  5
Yan Han                       5
Samohin Daniel                5
Chen Nathan                   5
Kerry Brendan                 5
Cha Junhwan                   5
Ge Misha                      5
Bychenko Alexei               5
Zhou Vincent                  5
Hendrickx Jorik               5
Messing Keegan                5
Brezina Michal                5
Kolyada Mikhail               5
Rippon Adam                   5
Chan Patrick                  5
Aliev Dmitri                  5
Jin Boyang                    5
Uno Shoma                     5
Paniot Yaroslav               5
Name: count, dtype: int64

In [125]:
(
    program_df["name"].value_counts()
    .reset_index()
    ["count"].unique()
)

array([5])

In [126]:
program_df.head(10)

,rank,name,noc,starting_number,tss,tes,tpcs,deductions,program_component,factor,final_score,J1,J2,J3,J4,J5,J6,J7,J8,J9
3,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,Composition,1.0000,9.7500,9.5000,10.0000,9.7500,9.7500,9.7500,9.7500,9.5000,10.0000,9.7500
4,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,Interpretation of the Music,1.0000,9.7500,9.5000,9.7500,9.7500,10.0000,9.7500,9.7500,9.5000,10.0000,9.7500
2,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,Performance,1.0000,9.8600,10.0000,9.7500,10.0000,10.0000,9.7500,9.7500,9.7500,10.0000,9.7500
0,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,Skating Skills,1.0000,9.7100,10.0000,9.7500,10.0000,9.7500,9.7500,9.5000,9.5000,9.7500,9.5000
1,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,Transitions,1.0000,9.4300,9.0000,9.5000,9.7500,9.5000,9.5000,9.2500,8.7500,9.7500,9.5000
8,2,Fernandez Javier,ESP,29,107.5800,59.7900,47.7900,0.0000,Composition,1.0000,9.6800,10.0000,10.0000,9.5000,10.0000,9.5000,9.7500,9.2500,9.5000,9.5000
9,2,Fernandez Javier,ESP,29,107.5800,59.7900,47.7900,0.0000,Interpretation of the Music,1.0000,9.6800,10.0000,9.7500,10.0000,9.7500,9.5000,9.7500,9.5000,9.5000,9.2500
7,2,Fernandez Javier,ESP,29,107.5800,59.7900,47.7900,0.0000,Performance,1.0000,9.7100,10.0000,9.7500,10.0000,10.0000,9.2500,9.5000,9.5000,9.7500,9.5000
5,2,Fernandez Javier,ESP,29,107.5800,59.7900,47.7900,0.0000,Skating Skills,1.0000,9.3600,10.0000,9.5000,9.5000,9.5000,9.2500,9.5000,9.0000,9.2500,9.0000
6,2,Fernandez Javier,ESP,29,107.5800,59.7900,47.7900,0.0000,Transitions,1.0000,9.3600,9.5000,9.5000,9.5000,9.5000,9.0000,9.5000,9.0000,9.2500,9.2500


In [127]:
program_df.tail(10)

,rank,name,noc,starting_number,tss,tes,tpcs,deductions,program_component,factor,final_score,J1,J2,J3,J4,J5,J6,J7,J8,J9
143,29,Montoya Felipe,ESP,1,52.4100,22.5900,30.8200,-1.0000,Composition,1.0000,6.2500,6.2500,6.0000,6.0000,6.0000,6.5000,6.0000,7.0000,7.0000,6.0000
144,29,Montoya Felipe,ESP,1,52.4100,22.5900,30.8200,-1.0000,Interpretation of the Music,1.0000,6.2500,6.5000,6.2500,6.0000,6.2500,6.2500,5.7500,7.2500,6.5000,6.0000
142,29,Montoya Felipe,ESP,1,52.4100,22.5900,30.8200,-1.0000,Performance,1.0000,6.0000,6.0000,5.7500,5.7500,5.7500,6.2500,5.5000,7.0000,6.7500,5.7500
140,29,Montoya Felipe,ESP,1,52.4100,22.5900,30.8200,-1.0000,Skating Skills,1.0000,6.2500,6.2500,6.2500,6.0000,6.0000,6.5000,6.0000,6.7500,6.7500,6.0000
141,29,Montoya Felipe,ESP,1,52.4100,22.5900,30.8200,-1.0000,Transitions,1.0000,6.0700,6.5000,6.0000,5.7500,5.7500,6.0000,6.0000,7.0000,6.5000,5.7500
148,30,Paniot Yaroslav,UKR,7,46.5800,18.6800,29.9000,-2.0000,Composition,1.0000,6.2100,6.5000,6.2500,5.0000,6.0000,7.0000,5.7500,7.5000,6.0000,6.0000
149,30,Paniot Yaroslav,UKR,7,46.5800,18.6800,29.9000,-2.0000,Interpretation of the Music,1.0000,5.8600,6.0000,6.0000,5.0000,5.7500,7.2500,5.5000,7.0000,5.0000,5.7500
147,30,Paniot Yaroslav,UKR,7,46.5800,18.6800,29.9000,-2.0000,Performance,1.0000,5.6100,5.5000,5.7500,4.7500,5.5000,6.5000,5.2500,6.7500,5.2500,5.5000
145,30,Paniot Yaroslav,UKR,7,46.5800,18.6800,29.9000,-2.0000,Skating Skills,1.0000,6.4300,7.0000,6.5000,5.0000,6.2500,7.0000,6.0000,6.5000,6.5000,6.2500
146,30,Paniot Yaroslav,UKR,7,46.5800,18.6800,29.9000,-2.0000,Transitions,1.0000,5.7900,6.7500,6.0000,4.7500,5.5000,6.0000,5.2500,6.5000,5.5000,5.7500


In [128]:
program_df["starting_number"].value_counts().reset_index().sort_values(by="starting_number")

,starting_number,count
2,1,5
5,2,5
19,3,5
4,4,5
8,5,5
3,6,5
29,7,5
12,8,5
6,9,5
15,10,5


## Merging Element Components with Program Components

In [129]:
element_df_renamed = (
                        element_df
                            .set_index(
                                ['rank', 'name', 'noc', 'starting_number', 'tss', 'tes', 'tpcs', 'deductions']
                                )
                    ) 

program_df_renamed = (
                        program_df
                            .set_index(
                                ['rank', 'name', 'noc', 'starting_number', 'tss', 'tes', 'tpcs', 'deductions']
                                )
                    ) 

data_df = pd.concat([element_df_renamed, program_df_renamed])

cols_at_end = ['J1', 'J2', 'J3', 'J4', 'J5', 'J6', 'J7', 'J8', 'J9']
cols_not_at_end = data_df.columns.difference(cols_at_end).to_list()
new_column_order = cols_not_at_end + cols_at_end
data_df = data_df[new_column_order]

data_df = (
    data_df
        .reset_index()
        .sort_values(by=['rank', 'name', 'noc', 'starting_number', 'tss', 'tes', 'tpcs', 'deductions'])
        .reset_index(drop=True)
)

## Sense Checks on Final data_df

In [130]:
data_df.shape

(360, 26)

In [131]:
# each participant should have 7 element + 5 program rows = 12 rows total
data_df["name"].value_counts().reset_index()["count"].unique()

array([12])

In [132]:
# there should be 29 participants
len(data_df["name"].unique())

30

In [133]:
data_df.head(12)

,rank,name,noc,starting_number,tss,tes,tpcs,deductions,base_value,element,element_no,extra_points,factor,final_score,goe,info,program_component,J1,J2,J3,J4,J5,J6,J7,J8,J9
0,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,10.5000,4S,1.0000,0.0000,NaN,13.2100,2.7100,None,NaN,3.0000,2.0000,3.0000,3.0000,3.0000,2.0000,3.0000,3.0000,2.0000
1,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,3.2000,FCSp4,2.0000,0.0000,NaN,4.2000,1.0000,None,NaN,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,3.0000,2.0000
2,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,3.0000,CSSp4,3.0000,0.0000,NaN,4.4300,1.4300,None,NaN,2.0000,3.0000,3.0000,3.0000,3.0000,2.0000,3.0000,3.0000,3.0000
3,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,9.3500,3A,4.0000,1.0000,NaN,12.3500,3.0000,None,NaN,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000
4,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,16.0600,4T+3T,5.0000,1.0000,NaN,18.6300,2.5700,None,NaN,3.0000,2.0000,2.0000,3.0000,2.0000,2.0000,3.0000,3.0000,3.0000
5,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,3.9000,StSq4,6.0000,0.0000,NaN,6.0000,2.1000,None,NaN,2.0000,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000
6,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,3.0000,CCoSp3,7.0000,0.0000,NaN,4.3600,1.3600,None,NaN,2.0000,3.0000,2.0000,3.0000,3.0000,3.0000,2.0000,3.0000,3.0000
7,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,NaN,NaN,NaN,NaN,1.0000,9.7500,NaN,NaN,Composition,9.5000,10.0000,9.7500,9.7500,9.7500,9.7500,9.5000,10.0000,9.7500
8,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,NaN,NaN,NaN,NaN,1.0000,9.7500,NaN,NaN,Interpretation of the Music,9.5000,9.7500,9.7500,10.0000,9.7500,9.7500,9.5000,10.0000,9.7500
9,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,NaN,NaN,NaN,NaN,1.0000,9.8600,NaN,NaN,Performance,10.0000,9.7500,10.0000,10.0000,9.7500,9.7500,9.7500,10.0000,9.7500


## Applying Data Pipeline to All Singles Files

### Helper Functions

In [134]:
skater_header_pattern = re.compile(
        r"""
        ^(\d+)\s+                  # 1 rank
        (.+?)\s+                   # 2 name
        ([A-Z]{3})\s+              # 3 NOC code
        (\d+)\s+                   # 4 starting num
        (\d+\.\d{2})\s+            # 5 total segment score
        (\d+\.\d{2})\s+            # 6 total element score
        (\d+\.\d{2})\s+            # 7 total program score
        (-?\d+\.\d{2})$            # 8 total deductions
        """,
        re.VERBOSE | re.MULTILINE
    )

element_pattern = re.compile(
    r"""
    ^\s*(\d+)\s+                 # 1 element number
    ([A-Za-z0-9+!*<>q]+)\s+      # 2 element code
    (?:(\S+)\s+)?                # 3 optional info column (x, q, !, etc.)
    ([\d.]+)\s+                  # 4 base value
    (?:\b(x)\b\s+)?              # 5 optional extra points column (x)
    ([\-\d.]+)\s+                # 6 GOE
    ((?:(?:-?\d+)|-)(?:\s+(?:(?:-?\d+)|-)){8}\s+)  # 7 judges scores 
    ([\d.]+)$                    # 8 final score
    """,
    re.VERBOSE | re.MULTILINE
)

program_components_pattern = re.compile(
    r"""
    ^(Skating\s+Skills|Transitions|Performance|Composition|Interpretation\s+of\s+the\s+Music)\s+  # 1 component
    (\d+\.\d{2})\s+                     # 2 factor
    ((?:\d+\.\d{2}\s+){9})              # 3 judge scores
    (\d+\.\d{2})$                       # 4 final score
    """,
    re.VERBOSE | re.MULTILINE
)

In [135]:
def _get_full_pdf_text(pdf_path):
    pages_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                pages_text.append(text)

    full_text = "\n".join(pages_text)
    # full_text = full_text.replace(" x ", " ")

    return full_text

def _get_skater_blocks(full_text):
    matches = list(skater_header_pattern.finditer(full_text))
    skater_blocks = []
    for i, m in enumerate(matches):
        start = m.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(full_text)
        skater_blocks.append((m, full_text[start:end]))
    return skater_blocks

def _process_skater_block_element(header, block):
    # read header
    rank = int(header.group(1))
    name = header.group(2).title()
    noc = header.group(3)
    starting_number = int(header.group(4))
    tss = float(header.group(5))
    tes = float(header.group(6))
    tpcs = float(header.group(7))
    deductions = float(header.group(8))

    elements_rows = []
    for m in element_pattern.finditer(block):
        row = {
            "rank": rank,
            "name": name,
            "noc": noc,
            "starting_number": starting_number,
            "tss": tss,
            "tes": tes,
            "tpcs": tpcs,
            "deductions": deductions,
            "element_no": int(m.group(1)),
            "element": m.group(2),
            "info": m.group(3),
            "base_value": float(m.group(4)),
            "extra_points": 1 if m.group(5) else 0,
            "goe": float(m.group(6)),
            "final_score": float(m.group(8)),
        }

        judges = m.group(7).split()
        for i, j in enumerate(judges):
            try:
                row[f"J{i+1}"] = int(j)
            except ValueError:
                row[f"J{i+1}"] = 0
        
        elements_rows.append(row)

    return elements_rows

def _process_skater_block_program(header, block):
    # read header
    rank = int(header.group(1))
    name = header.group(2).title()
    noc = header.group(3)
    starting_number = int(header.group(4))
    tss = float(header.group(5))
    tes = float(header.group(6))
    tpcs = float(header.group(7))
    deductions = float(header.group(8))

    program_rows = []
    for m in program_components_pattern.finditer(block):
        row = {
            "rank": rank,
            "name": name,
            "noc": noc,
            "starting_number": starting_number,
            "tss": tss,
            "tes": tes,
            "tpcs": tpcs,
            "deductions": deductions,
            "program_component": m.group(1),
            "factor": float(m.group(2)),
            "final_score": float(m.group(4)),
        }

        judges = m.group(3).split()
        for i, j in enumerate(judges):
            try:
                row[f"J{i+1}"] = float(j)
            except ValueError:
                row[f"J{i+1}"] = 0
        
        program_rows.append(row)

    return program_rows

def parsing_fsk_score_sheet(pdf_path):
    full_text = _get_full_pdf_text(pdf_path)
    skater_blocks = _get_skater_blocks(full_text)

    elements_rows = []
    program_rows = []

    for header, block in skater_blocks:
        processed_rows = _process_skater_block_element(header, block)
        elements_rows.extend(processed_rows)
        
        processed_rows = _process_skater_block_program(header, block)
        program_rows.extend(processed_rows)

    element_df = pd.DataFrame(elements_rows)
    element_df.sort_values(["rank", "element_no"], inplace=True)

    program_df = pd.DataFrame(program_rows)
    program_df.sort_values(["rank", "program_component"], inplace=True)

    return element_df, program_df

def _add_file_features(pdf_path, data_df):
    is_short_program = 1 if "SP" in pdf_path else 0
    data_df["is_short_program"] = is_short_program

    category = "men" if "Men" in pdf_path else "women" if "Ladies" in pdf_path else "pairs" if "Pair" in pdf_path else ""
    data_df["category"] = category

    event_type = "team" if "Team" in pdf_path else "individual"
    data_df["event_type"] = event_type

    return data_df
    
    return data_df

def get_fsk_df(pdf_path):
    element_df, program_df = parsing_fsk_score_sheet(pdf_path)
    element_df_renamed = (
                        element_df
                            .set_index(
                                ['rank', 'name', 'noc', 'starting_number', 'tss', 'tes', 'tpcs', 'deductions']
                                )
                    ) 

    program_df_renamed = (
                            program_df
                                .set_index(
                                    ['rank', 'name', 'noc', 'starting_number', 'tss', 'tes', 'tpcs', 'deductions']
                                    )
                        ) 

    data_df = pd.concat([element_df_renamed, program_df_renamed])
    data_df = data_df.assign(
            year = isu_year,
            event = isu_event,
            is_element = lambda x: (~x.element_no.isna()).astype(int)
        )

    cols_at_end = ['J1', 'J2', 'J3', 'J4', 'J5', 'J6', 'J7', 'J8', 'J9']
    cols_not_at_end = data_df.columns.difference(cols_at_end).to_list()
    new_column_order = cols_not_at_end + cols_at_end
    data_df = data_df[new_column_order]

    data_df = (
        data_df
            .reset_index()
            .sort_values(by=['rank', 'name', 'noc', 'starting_number', 'tss', 'tes', 'tpcs', 'deductions'])
            .reset_index(drop=True)
    )
    data_df = _add_file_features(pdf_path, data_df)
    
    return data_df

### Testing Helpers

In [136]:
data_df = get_fsk_df(PDF_PATH)
data_df.head(30)

,rank,name,noc,starting_number,tss,tes,tpcs,deductions,base_value,element,element_no,event,extra_points,factor,final_score,goe,info,is_element,program_component,year,J1,J2,J3,J4,J5,J6,J7,J8,J9,is_short_program,category,event_type
0,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,10.5000,4S,1.0000,owg2018,0.0000,NaN,13.2100,2.7100,None,1,NaN,1718,3.0000,2.0000,3.0000,3.0000,3.0000,2.0000,3.0000,3.0000,2.0000,1,men,individual
1,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,3.2000,FCSp4,2.0000,owg2018,0.0000,NaN,4.2000,1.0000,None,1,NaN,1718,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,3.0000,2.0000,1,men,individual
2,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,3.0000,CSSp4,3.0000,owg2018,0.0000,NaN,4.4300,1.4300,None,1,NaN,1718,2.0000,3.0000,3.0000,3.0000,3.0000,2.0000,3.0000,3.0000,3.0000,1,men,individual
3,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,9.3500,3A,4.0000,owg2018,1.0000,NaN,12.3500,3.0000,None,1,NaN,1718,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000,1,men,individual
4,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,16.0600,4T+3T,5.0000,owg2018,1.0000,NaN,18.6300,2.5700,None,1,NaN,1718,3.0000,2.0000,2.0000,3.0000,2.0000,2.0000,3.0000,3.0000,3.0000,1,men,individual
5,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,3.9000,StSq4,6.0000,owg2018,0.0000,NaN,6.0000,2.1000,None,1,NaN,1718,2.0000,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000,3.0000,1,men,individual
6,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,3.0000,CCoSp3,7.0000,owg2018,0.0000,NaN,4.3600,1.3600,None,1,NaN,1718,2.0000,3.0000,2.0000,3.0000,3.0000,3.0000,2.0000,3.0000,3.0000,1,men,individual
7,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,NaN,NaN,NaN,owg2018,NaN,1.0000,9.7500,NaN,NaN,0,Composition,1718,9.5000,10.0000,9.7500,9.7500,9.7500,9.7500,9.5000,10.0000,9.7500,1,men,individual
8,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,NaN,NaN,NaN,owg2018,NaN,1.0000,9.7500,NaN,NaN,0,Interpretation of the Music,1718,9.5000,9.7500,9.7500,10.0000,9.7500,9.7500,9.5000,10.0000,9.7500,1,men,individual
9,1,Hanyu Yuzuru,JPN,25,111.6800,63.1800,48.5000,0.0000,NaN,NaN,NaN,owg2018,NaN,1.0000,9.8600,NaN,NaN,0,Performance,1718,10.0000,9.7500,10.0000,10.0000,9.7500,9.7500,9.7500,10.0000,9.7500,1,men,individual


In [137]:
data_df[lambda x: x.name == "Lee Sihyeong"]

,rank,name,noc,starting_number,tss,tes,tpcs,deductions,base_value,element,element_no,event,extra_points,factor,final_score,goe,info,is_element,program_component,year,J1,J2,J3,J4,J5,J6,J7,J8,J9,is_short_program,category,event_type


In [138]:
data_df["name"].value_counts().reset_index()["count"].unique()

array([12])

### All Singles Data

In [139]:
from pathlib import Path

data_dfs_dict = {} # path to df dict
data_df = pd.DataFrame()

file_names = [item.name for item in Path(dir_name).iterdir() if item.is_file()]
valid_singles_files = []
skip = False
output_file_path = ""
for f in file_names:
    if ".csv" in f:
        skip = True
        output_file_path = dir_name + "/" +f
        break
if skip:
    print(f"Data Directory: {dir_name} already processed")
    data_df = pd.read_csv(output_file_path)
else:
    for f in file_names:
        print(f"PROCESSING {f}...")
        if "dance" not in f.lower():
            print(f"VALID SCORE SHEET...")
            valid_singles_files.append(f)
            
            full_path = dir_name + "/" + f
            df = get_fsk_df(full_path)
            data_dfs_dict[f] = df
        print(f"DONE")
        print("-------------------")

PROCESSING OWG2018_TeamIceDance_SD_D_Scores.pdf...
DONE
-------------------
PROCESSING OWG2018_TeamMen_SP_M_Scores.pdf...
VALID SCORE SHEET...
DONE
-------------------
PROCESSING OWG2018_LadiesSingleSkating_FS_Scores.pdf...
VALID SCORE SHEET...
DONE
-------------------
PROCESSING OWG2018_TeamLadies_SP_F_Scores.pdf...
VALID SCORE SHEET...
DONE
-------------------
PROCESSING OWG2018_TeamIceDance_FD_D_Scores.pdf...
DONE
-------------------
PROCESSING OWG2018_IceDance_SD_Scores.pdf...
DONE
-------------------
PROCESSING OWG2018_MenSingleSkating_SP_Scores.pdf...
VALID SCORE SHEET...
DONE
-------------------
PROCESSING OWG2018_PairSkating_SP_Scores.pdf...
VALID SCORE SHEET...
DONE
-------------------
PROCESSING OWG2018_TeamPairs_SP_P_Scores.pdf...
VALID SCORE SHEET...
DONE
-------------------
PROCESSING OWG2018_IceDance_FD_Scores.pdf...
DONE
-------------------
PROCESSING OWG2018_LadiesSingleSkating_SP_Scores.pdf...
VALID SCORE SHEET...
DONE
-------------------
PROCESSING OWG2018_TeamPairs_F

In [140]:
len(data_dfs_dict)

12

In [141]:
if not skip:
    print(valid_singles_files[0])
    data_dfs_dict[valid_singles_files[0]].head()


OWG2018_TeamMen_SP_M_Scores.pdf


In [142]:
if not skip:
    data_dfs_dict[valid_singles_files[0]]["name"].value_counts()

In [143]:
if not skip:
    for f in valid_singles_files:
        data_df = pd.concat([data_df, data_dfs_dict[f]])
    data_df.reset_index(drop=True, inplace=True)

    # output csv
    data_df.to_csv(OUTPUT_CSV, index=False)

    print(f"Saved {len(data_df)} rows to {OUTPUT_CSV}")

Saved 2716 rows to ./owg2018/owg2018.csv


In [144]:
data_df.shape

(2716, 32)

In [145]:
data_df

,rank,name,noc,starting_number,tss,tes,tpcs,deductions,base_value,element,element_no,event,extra_points,factor,final_score,goe,info,is_element,program_component,year,J1,J2,J3,J4,J5,J6,J7,J8,J9,is_short_program,category,event_type
0,1,Uno Shoma,JPN,10,103.2500,56.6400,46.6100,0.0000,12.3000,4F,1.0000,owg2018,0.0000,NaN,9.4400,-2.8600,None,1,NaN,1718,-3.0000,-2.0000,-2.0000,-2.0000,-2.0000,-3.0000,-2.0000,-2.0000,-3.0000,1,men,team
1,1,Uno Shoma,JPN,10,103.2500,56.6400,46.6100,0.0000,3.2000,FCSp4,2.0000,owg2018,0.0000,NaN,4.4100,1.2100,None,1,NaN,1718,2.0000,2.0000,3.0000,3.0000,2.0000,2.0000,3.0000,2.0000,3.0000,1,men,team
2,1,Uno Shoma,JPN,10,103.2500,56.6400,46.6100,0.0000,3.3000,StSq3,3.0000,owg2018,0.0000,NaN,4.6600,1.3600,None,1,NaN,1718,3.0000,2.0000,3.0000,3.0000,2.0000,2.0000,3.0000,3.0000,3.0000,1,men,team
3,1,Uno Shoma,JPN,10,103.2500,56.6400,46.6100,0.0000,16.0600,4T+3T,4.0000,owg2018,1.0000,NaN,17.9200,1.8600,None,1,NaN,1718,1.0000,2.0000,2.0000,2.0000,1.0000,2.0000,3.0000,2.0000,2.0000,1,men,team
4,1,Uno Shoma,JPN,10,103.2500,56.6400,46.6100,0.0000,9.3500,3A,5.0000,owg2018,1.0000,NaN,11.6400,2.2900,None,1,NaN,1718,2.0000,2.0000,3.0000,3.0000,2.0000,2.0000,2.0000,2.0000,3.0000,1,men,team
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,16,Hocke Annika / Blommaert Ruben,GER,4,108.9400,53.7900,55.1500,0.0000,NaN,NaN,NaN,owg2018,NaN,1.6000,7.0000,NaN,NaN,0,Composition,1718,6.5000,7.2500,6.7500,6.7500,7.0000,7.2500,7.0000,7.7500,7.0000,0,pairs,individual
2712,16,Hocke Annika / Blommaert Ruben,GER,4,108.9400,53.7900,55.1500,0.0000,NaN,NaN,NaN,owg2018,NaN,1.6000,6.9300,NaN,NaN,0,Interpretation of the Music,1718,6.2500,7.2500,6.5000,6.7500,6.5000,7.2500,7.0000,7.5000,7.2500,0,pairs,individual
2713,16,Hocke Annika / Blommaert Ruben,GER,4,108.9400,53.7900,55.1500,0.0000,NaN,NaN,NaN,owg2018,NaN,1.6000,6.7900,NaN,NaN,0,Performance,1718,6.5000,7.0000,7.0000,6.5000,6.7500,7.2500,6.5000,7.5000,6.5000,0,pairs,individual
2714,16,Hocke Annika / Blommaert Ruben,GER,4,108.9400,53.7900,55.1500,0.0000,NaN,NaN,NaN,owg2018,NaN,1.6000,6.9300,NaN,NaN,0,Skating Skills,1718,6.5000,7.0000,7.0000,6.5000,7.0000,7.2500,6.7500,7.7500,7.0000,0,pairs,individual


In [146]:
data_df[lambda x: x.name == "Zagitova Alina"]

,rank,name,noc,starting_number,tss,tes,tpcs,deductions,base_value,element,element_no,event,extra_points,factor,final_score,goe,info,is_element,program_component,year,J1,J2,J3,J4,J5,J6,J7,J8,J9,is_short_program,category,event_type
137,2,Zagitova Alina,OAR,22,156.6500,81.6200,75.0300,0.0000,2.0000,ChSq1,1.0000,owg2018,0.0000,NaN,3.7000,1.7000,None,1,NaN,1718,2.0000,3.0000,3.0000,3.0000,2.0000,2.0000,3.0000,2.0000,2.0000,0,women,individual
138,2,Zagitova Alina,OAR,22,156.6500,81.6200,75.0300,0.0000,3.2000,FCSp4,2.0000,owg2018,0.0000,NaN,4.2700,1.0700,None,1,NaN,1718,2.0000,2.0000,3.0000,2.0000,2.0000,2.0000,2.0000,2.0000,3.0000,0,women,individual
139,2,Zagitova Alina,OAR,22,156.6500,81.6200,75.0300,0.0000,3.9000,StSq4,3.0000,owg2018,0.0000,NaN,5.7000,1.8000,None,1,NaN,1718,2.0000,2.0000,3.0000,3.0000,2.0000,3.0000,3.0000,3.0000,2.0000,0,women,individual
140,2,Zagitova Alina,OAR,22,156.6500,81.6200,75.0300,0.0000,6.6000,3Lz,4.0000,owg2018,1.0000,NaN,7.1000,0.5000,None,1,NaN,1718,1.0000,0.0000,2.0000,1.0000,0.0000,1.0000,0.0000,1.0000,1.0000,0,women,individual
141,2,Zagitova Alina,OAR,22,156.6500,81.6200,75.0300,0.0000,8.3600,2A+3T,5.0000,owg2018,1.0000,NaN,9.7600,1.4000,None,1,NaN,1718,2.0000,2.0000,2.0000,2.0000,1.0000,2.0000,2.0000,2.0000,2.0000,0,women,individual
142,2,Zagitova Alina,OAR,22,156.6500,81.6200,75.0300,0.0000,9.2400,3F+2T+2Lo,6.0000,owg2018,1.0000,NaN,10.4400,1.2000,None,1,NaN,1718,2.0000,2.0000,3.0000,2.0000,2.0000,1.0000,1.0000,1.0000,2.0000,0,women,individual
143,2,Zagitova Alina,OAR,22,156.6500,81.6200,75.0300,0.0000,2.7000,LSp4,7.0000,owg2018,0.0000,NaN,3.7700,1.0700,None,1,NaN,1718,2.0000,2.0000,3.0000,3.0000,1.0000,2.0000,2.0000,2.0000,2.0000,0,women,individual
144,2,Zagitova Alina,OAR,22,156.6500,81.6200,75.0300,0.0000,12.2100,3Lz+3Lo,8.0000,owg2018,1.0000,NaN,13.9100,1.7000,None,1,NaN,1718,2.0000,2.0000,3.0000,3.0000,2.0000,3.0000,2.0000,2.0000,3.0000,0,women,individual
145,2,Zagitova Alina,OAR,22,156.6500,81.6200,75.0300,0.0000,4.8400,3S,9.0000,owg2018,1.0000,NaN,6.2400,1.4000,None,1,NaN,1718,2.0000,2.0000,3.0000,2.0000,1.0000,2.0000,2.0000,2.0000,2.0000,0,women,individual
146,2,Zagitova Alina,OAR,22,156.6500,81.6200,75.0300,0.0000,5.8300,3F,10.0000,owg2018,1.0000,NaN,7.5300,1.7000,None,1,NaN,1718,3.0000,2.0000,3.0000,3.0000,3.0000,2.0000,2.0000,2.0000,2.0000,0,women,individual
